In [53]:
"""
#remettre le all_dataset
from scipy.sparse import csc_matrix
from scipy.spatial.distance import pdist, squareform
from mangaki.utils.svd import MangakiSVD
from mangaki.utils.values import rating_values
from mangaki.models import Rating
import pandas, random
import numpy as np

#source : from 'csv' or 'database' ?

#from csv
"""
#ratings = pandas.read_csv('../data/ratings.csv', header=None).as_matrix()	
#nb_users = ratings[:,0].max()+1
#nb_items = ratings[:,1].max()+1
#items = range(0,nb_items)

"""

#from database
A, B = [], []
for user_id, work_id, choice in Rating.objects.values_list('user_id', 'work_id', 'choice'):
	A.append([user_id, work_id])
	B.append(rating_values[choice])

nb_users = np.asarray(A)[:,0].max()+1
nb_items = np.asarray(A)[:,1].max()+1
items = range(0,nb_items)



#2 fonctions donnant une matrice de départ user*item
"""
#récupérer une matrice users*items avec les 1er nb_ratings de ratings.csv
#-soit SVD ayant pour rang rank	
#-soit matrice creuse ayant pour coeff les ratings (creuse[i][j]=rating de l'utilisateur i pour l'oeuvre j)
"""
def get_matrix_svd(rank, nb_ratings):
	X = ratings[:nb_ratings,0:2]
	Y = [rating_values[rating] for rating in ratings[:nb_ratings,2]]
	svd = MangakiSVD(rank) 
	svd.set_parameters(nb_users,nb_items)
	svd.fit(X,Y)
	return svd.VT



def get_matrix_csc(nb_ratings): 
		row = ratings[:,0]
		col = ratings[:,1]
		data = np.array([rating_values[rating] for rating in ratings[:,2]])
		creux = csc_matrix((data, (row, col)), shape=(nb_users, nb_items))
		creuse = creux.toarray()
		return creuse     


#2 fonctions calculant des matrices de similarité

#prendre en argument une metric car pas très intéressant car déjà tt fait en Python

#option = 'cosine' pr 'jaccard'
def similarity(matrix, option): # matrix: dimension nb_users x dimension nb_works
    return 1 - squareform(pdist(matrix, metric=option))
	
class MangakiUniform(object):
	
	def __init__(self, nb_points):
		self.nb_points = nb_points


	def sample_k(self,matrix_similarity):
		uniform_items = list(range(matrix_similarity.shape[0]))
		random.shuffle(uniform_items)
		return uniform_items[:self.nb_points]
    	
	def __str__(self):
		return 'uniform list, nb_points=%d' % self.nb_points

class MangakiDPP(object):

	def __init__(self, nb_points):
		self.nb_points = nb_points
		

	#n'a pas d'intérêt au final
    #TODO : delete
	#similarity_fn='jaccard' or 'coisine'
	#def fit(self, matrix, all_dataset=False): #obtention de la matrice de similarité
	#	matrix_fit=similarity(matrix, similarity_fn)
	#	return matrix_fit
    
    #thanks to mehdidc on github
      # tirer au hasard des points
    #def sample_k(items, similiarity, k, max_nb_iterations=1000, rng=np.random):
	def sample_k(items, L, k, max_nb_iterations=1000, rng=np.random):
	
	#Sample a list of k items from a DPP defined
	#by the similarity matrix L. The algorithm
	#is iterative and runs for max_nb_iterations.
	#The algorithm used is from
	#(Fast Determinantal Point Process Sampling withw
	#Application to Clustering, Byungkon Kang, NIPS 2013)
	
		initial = rng.choice(range(len(items)), size=k, replace=False)
		X = [False] * len(items)
		for i in initial:
			X[i] = True
		X = np.array(X)
		for i in range(max_nb_iterations):
			u = rng.choice(np.arange(len(items))[X])
			v = rng.choice(np.arange(len(items))[~X])
			Y = X.copy()
			Y[u] = False
			L_Y = L[Y, :]
			L_Y = L_Y[:, Y]
			L_Y_inv = np.linalg.inv(L_Y)

			c_v = L[v:v+1, :]
			c_v = c_v[:, v:v+1]
			b_v = L[Y, :]
			b_v = b_v[:, v:v+1]
			c_u = L[u:u+1, :]
			c_u = c_u[:, u:u+1]
			b_u = L[Y, :]
			b_u = b_u[:, u:u+1]

			p = min(1, c_v - np.dot(np.dot(b_v.T, L_Y_inv), b_v) /
	              (c_u - np.dot(np.dot(b_u.T, L_Y_inv.T), b_u)))
			if rng.uniform() <= p:
				X = Y[:]
				X[v] = True
			return np.array(items)[X]
      
	def __str__(self):
		return 'sample dpp list, nb_points=%d' % self.nb_points
   

# comparer : regarder la distance moyenne entre deux points, mesures de dispersion. Avoir un truc qui permet de faire le fit + lancer quelques milliers de samples et regarder la dispersion moyenne
#ajouter des options : quels para de comparaison utilisés ? Diamètre d'ordre r (1 ou 0 ? ), somme de pdist, déterminant du squareform(pdsit) ?


#(pdist(svd.VT[:,sampled_items].T)).sum()

"""
""""""
#distance_sample = []
#distance_uniform = []
#et ds la boucle : 
#distance_sample.append((pdist(svd.VT[:,sampled_items].T)).sum())
#distance_uniform.append((pdist(svd.VT[:,uniform_items].T)).sum())
#puis comparaison entre les deux listes d'une manière ou d'une autre
""""""
"""
""""""

#det de pdist : np.linalg.det(squareform(pdist(svd.VT[:,uniform_items].T, metric='cosine')))

#diamètre d'ordre r : (ici 1 pr l'instant) 
"""
#n = len(sampled_items)
#coefficient_sample = 2/(n*(n-1))*(pdist(svd.VT[:,sampled_items].T).sum())
"""




def compare(type_get_matrix, nb_points, nb_iterations, nb_ratings):
	results_uniform, results_sample_dpp=[], []
	if type_get_matrix=='svd':

		matrix = get_matrix_svd(10, nb_ratings)
	else :
		matrix = get_matrix_svd(10, nb_ratings)
	uniform = MangakiUniform(nb_points)
	dpp = MangakiDDP(matrix, similarity_fn='cosine')
	similarity = dpp.fit(matrix)
	indicateur = 0
	pb = 0
	while indicateur != nb_iterations:
    
		try:
			sampled_items = dpp.sample_k(items, similarity, nb_points)
        

		except np.linalg.linalg.LinAlgError as err:
			pb = 1
		if pb==0:
			indicateur = indicateur+1
        # cas où tt se passe bien, bloc où l'on exécute la comparaison
			uniform_items = uniform.sample_k(items, similarity, nb_points)
        	
			det_uni=np.linalg.det(squareform(pdist(svd.VT[:,uniform_items].T, metric='cosine')))
			det_dpp=np.linalg.det(squareform(pdist(svd.VT[:,sampled_items].T, metric='cosine')))
        	
			diam_uni = 2/(nb_points*(nb_points-1))*(pdist(svd.VT[:,uniform_items].T).sum())
			diam_dpp = 2/(nb_points*(nb_points-1))*(pdist(svd.VT[:,sampled_items].T).sum())

			results_uniform.append([det_uni, diam_uni]) #à compléter : det et diamètre d'ordre r
			results_sample_dpp.append([det_dpp, diam_dpp]) #à compléter : det et diamètre d'ordre r
		else :
			pb = 0
            
""""

In [8]:
#A
#B
#nb_users
#nb_items


In [4]:
#remettre le all_dataset
from scipy.sparse import csc_matrix
from scipy.spatial.distance import pdist, squareform
from mangaki.utils.svd import MangakiSVD
from mangaki.utils.values import rating_values
from mangaki.models import Rating
import pandas, random
import numpy as np

#source : from 'csv' or 'database' ?

#from csv
"""
ratings = pandas.read_csv('../data/ratings.csv', header=None).as_matrix()	
nb_users = ratings[:,0].max()+1
nb_items = ratings[:,1].max()+1
items = range(0,nb_items)

"""

#from database
X, Y = [], []
for user_id, work_id, choice in Rating.objects.values_list('user_id', 'work_id', 'choice'):
	X.append([user_id, work_id])
	Y.append(rating_values[choice])
X=np.asarray(X)
nb_users = np.asarray(X)[:,0].max()+1
nb_items = np.asarray(X)[:,1].max()+1
items = range(0,nb_items)



#2 fonctions donnant une matrice de départ user*item
"""
récupérer une matrice users*items avec les 1er nb_ratings de ratings.csv
-soit SVD ayant pour rang rank	
-soit matrice creuse ayant pour coeff les ratings (creuse[i][j]=rating de l'utilisateur i pour l'oeuvre j)
"""
def get_matrix_svd(rank, nb_ratings):
	Xpr = X[:nb_ratings,0:2]
	Ypr = Y[:nb_ratings]
	svd = MangakiSVD(rank) 
	svd.set_parameters(nb_users,nb_items)
	svd.fit(Xpr,Ypr)
	return svd.VT



def get_matrix_csc(nb_ratings): 
		row = X[:nb_ratings,0]
		col = X[:nb_ratings,1]
		data = np.array(Y[:nb_ratings])
		creux = csc_matrix((data, (row, col)), shape=(nb_users, nb_items))
		creuse = creux.toarray()
		return creuse     


#2 fonctions calculant des matrices de similarité

#prendre en argument une metric car pas très intéressant car déjà tt fait en Python

#option = 'cosine' pr 'jaccard'
def similarity(matrix, option): # matrix: dimension nb_users x dimension nb_works
    return 1 - squareform(pdist(matrix.T, metric=option))
	
class MangakiUniform(object):
	
	def __init__(self, nb_points):
		self.nb_points = nb_points


	def sample_k(self,matrix_similarity):
		uniform_items = list(range(matrix_similarity.shape[0]))
		random.shuffle(uniform_items)
		return uniform_items[:self.nb_points]
    	
	def __str__(self):
		return 'uniform list, nb_points=%d' % self.nb_points

class MangakiDPP(object):

	def __init__(self, nb_points):
		self.nb_points = nb_points


#n'a pas d'intérêt au final
#TODO : delete
#similarity_fn='jaccard' or 'coisine'
#def fit(self, matrix, all_dataset=False): #obtention de la matrice de similarité
#	matrix_fit=similarity(matrix, similarity_fn)
#	return matrix_fit
    
#thanks to mehdidc on github
# tirer au hasard des points
#def sample_k(items, similiarity, k, max_nb_iterations=1000, rng=np.random):
       #def sample_k(items, similiarity, k, max_nb_iterations=1000, rng=np.random):
	def sample_k(self, items, L, k, max_nb_iterations=1000, rng=np.random):
	
	#Sample a list of k items from a DPP defined
	#by the similarity matrix L. The algorithm
	#is iterative and runs for max_nb_iterations.
	#The algorithm used is from
	#(Fast Determinantal Point Process Sampling withw
	#Application to Clustering, Byungkon Kang, NIPS 2013)
	
		initial = rng.choice(range(len(items)), size=k, replace=False)
		X = [False] * len(items)
		for i in initial:
			X[i] = True
		X = np.array(X)
		for i in range(max_nb_iterations):
			u = rng.choice(np.arange(len(items))[X])
			v = rng.choice(np.arange(len(items))[~X])
			Y = X.copy()
			Y[u] = False
			L_Y = L[Y, :]
			L_Y = L_Y[:, Y]
			L_Y_inv = np.linalg.inv(L_Y)

			c_v = L[v:v+1, :]
			c_v = c_v[:, v:v+1]
			b_v = L[Y, :]
			b_v = b_v[:, v:v+1]
			c_u = L[u:u+1, :]
			c_u = c_u[:, u:u+1]
			b_u = L[Y, :]
			b_u = b_u[:, u:u+1]

			p = min(1, c_v - np.dot(np.dot(b_v.T, L_Y_inv), b_v) /
	              (c_u - np.dot(np.dot(b_u.T, L_Y_inv.T), b_u)))
			if rng.uniform() <= p:
				X = Y[:]
				X[v] = True
			return np.array(items)[X]
      
    
	def __str__(self):
		return 'sample dpp list, nb_points=%d' % self.nb_points
   

# comparer : regarder la distance moyenne entre deux points, mesures de dispersion. Avoir un truc qui permet de faire le fit + lancer quelques milliers de samples et regarder la dispersion moyenne
#ajouter des options : quels para de comparaison utilisés ? Diamètre d'ordre r (1 ou 0 ? ), somme de pdist, déterminant du squareform(pdsit) ?


#(pdist(svd.VT[:,sampled_items].T)).sum()

"""
distance_sample = []
distance_uniform = []
et ds la boucle : 
distance_sample.append((pdist(svd.VT[:,sampled_items].T)).sum())
distance_uniform.append((pdist(svd.VT[:,uniform_items].T)).sum())
puis comparaison entre les deux listes d'une manière ou d'une autre
"""


#det de pdist : np.linalg.det(squareform(pdist(svd.VT[:,uniform_items].T, metric='cosine')))

#diamètre d'ordre r : (ici 1 pr l'instant) 
"""
n = len(sampled_items)
coefficient_sample = 2/(n*(n-1))*(pdist(svd.VT[:,sampled_items].T).sum())
"""




def compare(type_get_matrix, nb_points, nb_iterations, nb_ratings):
	results_uniform, results_sample_dpp=[], []
	if type_get_matrix=='svd':

		matrix = get_matrix_svd(10, nb_ratings)
	else :
		matrix = get_matrix_svd(10, nb_ratings)
	uniform = MangakiUniform(nb_points)
	dpp = MangakiDDP(matrix, similarity_fn='cosine')
	similarity = dpp.fit(matrix)
	indicateur = 0
	pb = 0
	while indicateur != nb_iterations:
    
		try:
			sampled_items = dpp.sample_k(items, similarity, nb_points)
        

		except np.linalg.linalg.LinAlgError as err:
			pb = 1
		if pb==0:
			indicateur = indicateur+1
        # cas où tt se passe bien, bloc où l'on exécute la comparaison
			uniform_items = uniform.sample_k(items, similarity, nb_points)
        	
			det_uni=np.linalg.det(squareform(pdist(svd.VT[:,uniform_items].T, metric='cosine')))
			det_dpp=np.linalg.det(squareform(pdist(svd.VT[:,sampled_items].T, metric='cosine')))
        	
			diam_uni = 2/(nb_points*(nb_points-1))*(pdist(svd.VT[:,uniform_items].T).sum())
			diam_dpp = 2/(nb_points*(nb_points-1))*(pdist(svd.VT[:,sampled_items].T).sum())

			results_uniform.append([det_uni, diam_uni]) #à compléter : det et diamètre d'ordre r
			results_sample_dpp.append([det_dpp, diam_dpp]) #à compléter : det et diamètre d'ordre r
		else :
			pb = 0

In [2]:
X

array([[2087,    1],
       [1994,    2],
       [1994,    3],
       ..., 
       [2026, 1384],
       [2026, 9013],
       [2026, 2257]])

In [4]:
#Y

In [3]:
nb_users

2100

In [4]:
nb_items

14416

In [6]:
matrix=get_matrix_svd(10)
matrix

Computing M: (2100 × 14416)
fill and center matrix [349 ms]
Shapes (2100, 10) (10,) (10, 14416)
factor matrix [64325 ms]


array([[ -4.76171693e-16,   1.01020483e-01,   5.93017784e-02, ...,
         -0.00000000e+00,  -9.44467650e-05,  -1.60743802e-04],
       [ -4.68550127e-15,  -1.83381462e-02,  -7.07910668e-03, ...,
         -0.00000000e+00,  -1.13539014e-05,  -2.08125466e-03],
       [  8.86429943e-14,  -9.75645378e-02,  -2.02313541e-03, ...,
          0.00000000e+00,   1.00975773e-04,  -2.50413527e-03],
       ..., 
       [ -9.62286895e-11,   9.94452876e-03,   4.81962899e-02, ...,
          0.00000000e+00,   1.33213309e-04,   1.70693862e-02],
       [ -2.70149909e-12,   4.34641981e-02,  -6.43504981e-02, ...,
         -0.00000000e+00,  -2.40416928e-04,  -2.28896131e-03],
       [  4.08080827e-10,   1.52707993e-02,   1.04863916e-01, ...,
          0.00000000e+00,   3.21102824e-04,  -6.47039993e-03]])

In [21]:
matrix2=get_matrix_csc(len(Y))
get_matrix_csc(len(Y))

TypeError: get_matrix_csc() takes 0 positional arguments but 1 was given

In [22]:
np.shape(matrix)

(10, 14416)

In [20]:
similarity1=similarity(matrix,'cosine')
similarity1
np.shape(similarity1)

(10, 10)

In [10]:
#nan ...
similarity(matrix2,'cosine')

array([[ 1.        ,         nan,         nan, ...,         nan,
                nan,         nan],
       [        nan,  1.        ,         nan, ...,         nan,
                nan,         nan],
       [        nan,         nan,  1.        , ...,         nan,
                nan,         nan],
       ..., 
       [        nan,         nan,         nan, ...,  1.        ,
         0.31942857,  0.48542846],
       [        nan,         nan,         nan, ...,  0.31942857,
         1.        ,  0.31879469],
       [        nan,         nan,         nan, ...,  0.48542846,
         0.31879469,  1.        ]])

In [98]:
similarity(matrix,'jaccard')#pb avec les ratings_values 
#à corriger ............

array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]])

In [ ]:
#mauvaise idée
#similarity(matrix2,'jaccard')

In [29]:
uniform_test=MangakiUniform(10)
uniform_test.sample_k(similarity1)


[3648, 2514, 8126, 3155, 5299, 3170, 467, 3329, 4357, 167]

In [17]:
items = range(0,nb_items)
dpp_test=MangakiDPP(10)
dpp_test.sample_k(items,similarity1,10)

/home/voisin/mangaki/venv/lib/python3.4/site-packages/ipykernel/__main__.py:116: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 10 but corresponding boolean dimension is 14416



IndexError: index 660 is out of bounds for axis 0 with size 10

In [108]:
from scipy.spatial.distance import pdist, squareform

def compute_similarity_coisine(SVD):
    return 1 - squareform(pdist(SVD.T, metric='cosine'))

similarity=compute_similarity_coisine()

NameError: name 'svd' is not defined

In [105]:
def sample_k(items, L, k, max_nb_iterations=1000, rng=np.random):
    """
    Sample a list of k items from a DPP defined
    by the similarity matrix L. The algorithm
    is iterative and runs for max_nb_iterations.
    The algorithm used is from
    (Fast Determinantal Point Process Sampling withw
    Application to Clustering, Byungkon Kang, NIPS 2013)
    """
    initial = rng.choice(range(len(items)), size=k, replace=False)
    X = [False] * len(items)
    for i in initial:
        X[i] = True
    X = np.array(X)
    for i in range(max_nb_iterations):
        u = rng.choice(np.arange(len(items))[X])
        v = rng.choice(np.arange(len(items))[~X])
        Y = X.copy()
        Y[u] = False
        L_Y = L[Y, :]
        L_Y = L_Y[:, Y]
        L_Y_inv = np.linalg.inv(L_Y)

        c_v = L[v:v+1, :]
        c_v = c_v[:, v:v+1]
        b_v = L[Y, :]
        b_v = b_v[:, v:v+1]
        c_u = L[u:u+1, :]
        c_u = c_u[:, u:u+1]
        b_u = L[Y, :]
        b_u = b_u[:, u:u+1]

        p = min(1, c_v - np.dot(np.dot(b_v.T, L_Y_inv), b_v) /
                (c_u - np.dot(np.dot(b_u.T, L_Y_inv.T), b_u)))
        if rng.uniform() <= p:
            X = Y[:]
            X[v] = True
    return np.array(items)[X]

In [1]:
sample_k(items,similarity1,10)


NameError: name 'sample_k' is not defined

In [21]:
from mangaki.utils.svd import MangakiSVD
import numpy as np
import pandas 
from mangaki.utils.values import rating_values
ratings = pandas.read_csv('../data/ratings.csv', header=None).as_matrix()	
nb_users = ratings[:,0].max()+1
nb_items = ratings[:,1].max()+1
items = range(0,nb_items)
items


range(0, 8347)

In [19]:
svd =MangakiSVD(10)
X=ratings[:,0:2]
Y=[rating_values[rating] for rating in ratings[:,2]]
nb_users=X[:,0].max()+1
nb_items=X[:,1].max()+1
svd.set_parameters(nb_users,nb_items)
svd.fit(X,Y)

Computing M: (1710 × 8347)
fill and center matrix [1223 ms]
Shapes (1710, 10) (10,) (10, 8347)
factor matrix [28501 ms]


In [16]:
matrix=svd.VT
similarity1=similarity(matrix,'cosine')
similarity1
np.shape(similarity1)

(8347, 8347)

In [ ]:
similarity

In [17]:
uniform_test=MangakiUniform(10)
uniform_test.sample_k(similarity1)

[1002, 6276, 3214, 749, 479, 7350, 8194, 880, 1092, 5265]

In [23]:
items = range(0,nb_items)
dpp_test=MangakiDPP(10)
dpp_test.sample_k(items,similarity1,10)


array([  13,  677,  886, 2217, 5185, 5411, 5592, 6072, 6292, 7165])

In [34]:
results_uniform, results_sample_dpp=[], []

nb_iterations=1000
nb_points=10

indicateur = 0
pb = 0
while indicateur != nb_iterations:
    
	try:
		sampled_items = dpp_test.sample_k(items, similarity1, nb_points)
        

	except np.linalg.linalg.LinAlgError as err:
		pb = 1
	if pb==0:
		indicateur = indicateur+1
        # cas où tt se passe bien, bloc où l'on exécute la comparaison
		uniform_items = uniform_test.sample_k(similarity1)
        	
		det_uni=np.linalg.det(squareform(pdist(svd.VT[:,uniform_items].T, metric='cosine')))
		det_dpp=np.linalg.det(squareform(pdist(svd.VT[:,sampled_items].T, metric='cosine')))
        	
		diam_uni = 2/(nb_points*(nb_points-1))*(pdist(svd.VT[:,uniform_items].T).sum())
		diam_dpp = 2/(nb_points*(nb_points-1))*(pdist(svd.VT[:,sampled_items].T).sum())

		results_uniform.append([det_uni, diam_uni]) #à compléter : det et diamètre d'ordre r
		results_sample_dpp.append([det_dpp, diam_dpp]) #à compléter : det et diamètre d'ordre r
	else :
		pb = 0
    

In [36]:
results_uniform[:10]



[[-0.0010697073094202299, 0.019985279712718632],
 [0.0, 0.017093562650982972],
 [0.0, 0.014194468949938026],
 [-0.00050062360734603887, 0.052535853203791602],
 [-0.012483029388253493, 0.026240200979621077],
 [-0.019438381268891358, 0.047878581984790239],
 [0.0, 0.033863021726689729],
 [-6.5012795131765482e-07, 0.027729163930426928],
 [nan, 0.017394497854456716],
 [-0.00086948202085870212, 0.035598514641724276]]

In [38]:
results_sample_dpp[:10]

[[1.5447003990362022e-23, 0.01491703199029361],
 [-0.00063782807716318871, 0.018012786525769924],
 [-1.6328876558204219e-05, 0.017274596818214524],
 [-0.00090210557893651369, 0.013842650166151079],
 [-0.00034127366989724213, 0.030563847391876245],
 [-3.4092771508284516e-05, 0.024015104831022475],
 [-0.00031771831225448506, 0.01962054171020635],
 [-8.2099903564533512e-05, 0.0066761355590561388],
 [-0.00040741906229073356, 0.022249292971728788],
 [-1.1577956828292742e-09, 0.015735837614340119]]

In [47]:
np.asarray(results_sample_dpp)[:10,0]
np.asarray(results_sample_dpp)[0,0]

1.5447003990362022e-23

In [49]:
sum_det_sample=0
for i in range(0,nb_iterations):
    sum_det_sample=sum_det_sample+np.asarray(results_sample_dpp)[i,0]
sum_det_sample

nan

In [50]:
sum_det_uni=0
for i in range(0,nb_iterations):
    sum_det_uni=sum_det_uni+np.asarray(results_uniform)[i,0]

sum_det_uni

nan

In [52]:
sum_diam_sample=0
for i in range(0,nb_iterations):
    sum_diam_sample=sum_diam_sample+np.asarray(results_sample_dpp)[i,1]
sum_diam_sample

31.155883169098693

In [55]:
sum_diam_uni=0
for i in range(0,nb_iterations):
    sum_diam_uni=sum_diam_uni+np.asarray(results_uniform)[i,1]

sum_diam_uni

30.124278998810983

In [57]:
#vérif bien le bon nombre d'itérations 
results_uniform, results_sample_dpp=[], []

nb_iterations=5
nb_points=10

indicateur = 0
pb = 0
while indicateur != nb_iterations:
    
	try:
		sampled_items = dpp_test.sample_k(items, similarity1, nb_points)
        

	except np.linalg.linalg.LinAlgError as err:
		pb = 1
	if pb==0:
		indicateur = indicateur+1
        # cas où tt se passe bien, bloc où l'on exécute la comparaison
		uniform_items = uniform_test.sample_k(similarity1)
        	
		det_uni=np.linalg.det(squareform(pdist(svd.VT[:,uniform_items].T, metric='cosine')))
		det_dpp=np.linalg.det(squareform(pdist(svd.VT[:,sampled_items].T, metric='cosine')))
        	
		diam_uni = 2/(nb_points*(nb_points-1))*(pdist(svd.VT[:,uniform_items].T).sum())
		diam_dpp = 2/(nb_points*(nb_points-1))*(pdist(svd.VT[:,sampled_items].T).sum())

		results_uniform.append([det_uni, diam_uni]) #à compléter : det et diamètre d'ordre r
		results_sample_dpp.append([det_dpp, diam_dpp]) #à compléter : det et diamètre d'ordre r
	else :
		pb = 0
    

In [59]:
results_uniform


[[-0.0027586541846090946, 0.060292799006848224],
 [-4.5460223621872059e-05, 0.037927712594586639],
 [-0.049792730748434391, 0.047527250550762144],
 [-1.3064415931875985e-05, 0.02791814545724362],
 [nan, 0.016465710002629003]]

In [60]:
results_sample_dpp

[[-3.3195959025805404e-06, 0.040052104648133892],
 [-0.0046955933288646652, 0.027094018270038822],
 [-6.9400692878438767e-05, 0.022760504324026567],
 [-0.0027175927281073839, 0.023634232419756818],
 [-0.0001888625200747879, 0.054839556397156999]]